In [193]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout,MaxPooling1D, Flatten,BatchNormalization, GaussianNoise,Conv1D
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.utils import compute_class_weight
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, save_model, load_model

In [194]:
#this was created in general/diagnosis_making notebook
diag = pd.read_csv("../general/ground_truth.csv").drop("Unnamed: 0", axis=1)

Below we are combining several clinical datasets.

In [195]:
demo = pd.read_csv("PTDEMOG.csv").rename(columns={"PHASE":"Phase"})

In [196]:
neuro = pd.read_csv("NEUROEXM.csv").rename(columns={"PHASE":"Phase"})

In [197]:
neuro.columns

Index(['Phase', 'PTID', 'RID', 'VISCODE', 'VISCODE2', 'VISDATE', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 'NXOTHER',
       'NXABNORM', 'ID', 'SITEID', 'USERDATE', 'USERDATE2',
       'DD_CRF_VERSION_LABEL', 'LANGUAGE_CODE', 'HAS_QC_ERROR',
       'update_stamp'],
      dtype='object')

In [198]:
clinical = pd.read_csv("ADSP_PHC_COGN.csv").rename(columns={"PHASE":"Phase"})

In [199]:
clinical.head()

,RID,SUBJECT_KEY,Phase,VISCODE,VISCODE2,EXAMDATE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,AGE,DX,PHC_MEM,PHC_EXF,PHC_LAN,PHC_VSP,update_stamp
0,2,ADNI_011_S_0002,ADNI2,v06,m72,2011-09-19,1,16.0,2,5,80.4682,1.0,0.090,0.002,0.460,0.264,2021-04-14 13:05:55.0
1,2,ADNI_011_S_0002,ADNI2,v21,m96,2013-09-09,1,16.0,2,5,82.4422,1.0,0.288,-0.495,0.460,NaN,2021-04-14 13:05:56.0
2,2,ADNI_011_S_0002,ADNI1,m36,m36,2008-08-27,1,16.0,2,5,77.4073,1.0,0.344,0.419,0.535,NaN,2021-04-14 13:05:56.0
3,2,ADNI_011_S_0002,ADNI2,v41,m120,2015-09-22,1,16.0,2,5,84.4764,1.0,0.382,-0.549,0.595,-0.333,2021-04-14 13:05:56.0
4,2,ADNI_011_S_0002,ADNIGO,m60,m60,2010-09-22,1,16.0,2,5,79.4771,1.0,0.101,0.066,0.620,0.264,2021-04-14 13:05:56.0


In [200]:
diag["Subject"].value_counts()

023_S_0031     4
123_S_1300     4
131_S_0384     4
128_S_0272     4
123_S_0298     4
              ..
130_S_10216    1
135_S_10204    1
130_S_10214    1
033_S_10217    1
016_S_6836     1
Name: Subject, Length: 2589, dtype: int64

In [201]:
comb = pd.read_csv("ADSP_PHC_COGN.csv").rename(columns={"PHASE":"Phase"})
comb["PTID"]= comb["SUBJECT_KEY"].str.replace("ADNI_", "").str.replace("s", "S")
comb = comb[["RID", "PTID" , "Phase"]]

In [202]:
m = comb.merge(demo, on = ["RID", "Phase"]).merge(neuro,on = ["RID", "Phase"]).merge(clinical,on = ["RID", "Phase"]).drop_duplicates()

In [203]:
m.columns = [c[:-2] if str(c).endswith(('_x','_y')) else c for c in m.columns]

m = m.loc[:,~m.columns.duplicated()]

In [204]:
diag = diag.rename(columns = {"Subject": "PTID"})

In [205]:
m = m.merge(diag, on = ["PTID", "Phase"])

In [206]:
m["PTID"].value_counts()

131_S_0384    15
141_S_0697    15
003_S_1122    14
094_S_1417    14
067_S_4072    14
              ..
109_S_2278     1
137_S_0459     1
016_S_2284     1
127_S_0622     1
022_S_1097     1
Name: PTID, Length: 471, dtype: int64

In [207]:
t = m
pd.set_option('display.max_columns', None)
print(t)

       RID        PTID  Phase VISCODE VISCODE2     VISDATE  PTSOURCE  \
0       21  011_S_0021  ADNI1      sc       sc  2005-09-28         1   
1       21  011_S_0021  ADNI1      sc       sc  2005-09-28         1   
2       21  011_S_0021  ADNI1      sc       sc  2005-09-28         1   
3       21  011_S_0021  ADNI1      sc       sc  2005-09-28         1   
4       21  011_S_0021  ADNI1      sc       sc  2005-09-28         1   
...    ...         ...    ...     ...      ...         ...       ...   
2411  4659  021_S_4659  ADNI2     v01       sc  2012-04-03         1   
2412  4659  021_S_4659  ADNI2     v01       sc  2012-04-03         1   
2413  4659  021_S_4659  ADNI2     v01       sc  2012-04-03         1   
2414  4659  021_S_4659  ADNI2     v01       sc  2012-04-03         1   
2415  4659  021_S_4659  ADNI2     v01       sc  2012-04-03         1   

      PTGENDER    PTDOB  PTDOBYY  PTHAND  PTMARRY  PTEDUCAT  PTWORKHS  PTWORK  \
0          2.0  02/1933   1933.0     1.0      3.0     

In [208]:
t = t.drop(["ID",  "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2",
            "update_stamp",  "PTSOURCE", "PTDOB","DX"], axis=1) 

In [209]:
t.columns

Index(['RID', 'PTID', 'Phase', 'VISDATE', 'PTGENDER', 'PTDOBYY', 'PTHAND',
       'PTMARRY', 'PTEDUCAT', 'PTWORKHS', 'PTWORK', 'PTNOTRT', 'PTRTYR',
       'PTHOME', 'PTTLANG', 'PTPLANG', 'PTADBEG', 'PTCOGBEG', 'PTADDX',
       'PTETHCAT', 'PTRACCAT', 'PTIDENT', 'PTORIENT', 'PTORIENTOT', 'PTENGSPK',
       'PTNLANG', 'PTENGSPKAGE', 'PTCLANG', 'PTLANGSP', 'PTLANGWR', 'PTSPTIM',
       'PTSPOTTIM', 'PTLANGPR1', 'PTLANGSP1', 'PTLANGRD1', 'PTLANGWR1',
       'PTLANGUN1', 'PTLANGPR2', 'PTLANGSP2', 'PTLANGRD2', 'PTLANGWR2',
       'PTLANGUN2', 'PTLANGPR3', 'PTLANGSP3', 'PTLANGRD3', 'PTLANGWR3',
       'PTLANGUN3', 'PTLANGPR4', 'PTLANGSP4', 'PTLANGRD4', 'PTLANGWR4',
       'PTLANGUN4', 'PTLANGPR5', 'PTLANGSP5', 'PTLANGRD5', 'PTLANGWR5',
       'PTLANGUN5', 'PTLANGPR6', 'PTLANGSP6', 'PTLANGRD6', 'PTLANGWR6',
       'PTLANGUN6', 'PTLANGTTL', 'PTETHCATH', 'PTASIAN', 'PTOPI', 'PTBORN',
       'PTBIRPL', 'PTIMMAGE', 'PTIMMWHY', 'PTBIRPR', 'PTBIRGR',
       'DD_CRF_VERSION_LABEL', 'LANGUAGE_CODE', '

In [210]:
t = t.fillna(-4)
t = t.replace("-4", -4)
cols_to_delete = t.columns[(t == -4).sum()/len(t) > .70]
t.drop(cols_to_delete, axis = 1, inplace = True)

In [211]:
len(t.columns)
t.columns

Index(['RID', 'PTID', 'Phase', 'VISDATE', 'PTGENDER', 'PTDOBYY', 'PTHAND',
       'PTMARRY', 'PTEDUCAT', 'PTNOTRT', 'PTRTYR', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTCOGBEG', 'PTADDX', 'PTETHCAT', 'PTRACCAT', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 'NXOTHER',
       'NXABNORM', 'SUBJECT_KEY', 'EXAMDATE', 'AGE', 'PHC_MEM', 'PHC_EXF',
       'PHC_LAN', 'PHC_VSP', 'Group'],
      dtype='object')

In [212]:
# t["PTWORK"] = t["PTWORK"].str.lower().str.replace("housewife", "homemaker").str.replace("rn", "nurse").str.replace("bookeeper","bookkeeper").str.replace("cpa", "accounting")

In [213]:
# t["PTWORK"] = t["PTWORK"].fillna("-4").astype(str)

In [214]:
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*teach.*$)', 'education')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*bookkeep.*$)', 'bookkeeper')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*wife.*$)', 'homemaker')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*educat.*$)', 'education')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*engineer.*$)', 'engineer')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*eingineering.*$)', 'engineer') 
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*computer programmer.*$)', 'engineer') 
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*nurs.*$)', 'nurse')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*manage.*$)', 'managment')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*therapist.*$)', 'therapist')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*sales.*$)', 'sales')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*admin.*$)', 'admin')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*account.*$)', 'accounting')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*real.*$)', 'real estate')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*secretary.*$)', 'secretary')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*professor.*$)', 'professor')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*chem.*$)', 'chemist')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*business.*$)', 'business')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*writ.*$)', 'writing')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*psych.*$)', 'psychology')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*analys.*$)', 'analyst')

In [215]:
# cond = t['PTWORK'].value_counts()
# threshold = 10
# t['PTWORK'] = np.where(t['PTWORK'].isin(cond.index[cond >= threshold ]), t['PTWORK'], 'other')

In [216]:
# categorical = ['PTGENDER', 'PTWORK',
#  'PTHOME',
#  'PTMARRY',
#  'PTEDUCAT',
#  'PTPLANG',
#  'NXVISUAL',
#  'PTNOTRT',
#  'NXTREMOR',
#  'NXAUDITO',
#  'PTHAND']

# REMOVE PTWORK FOR NOW
categorical = ['PTGENDER',
 'PTHOME',
 'PTMARRY',
 'PTEDUCAT',
 'PTPLANG',
 'NXVISUAL',
 'PTNOTRT',
 'NXTREMOR',
 'NXAUDITO',
 'PTHAND']

In [217]:
quant = ['PTDOBYY',
 'PHC_MEM',
 'PHC_EXF',
 'PTRACCAT',
 'AGE',
 'PTADDX',
 'PTETHCAT',
 'PTCOGBEG',
 'PHC_VSP',
 'PHC_LAN']

In [218]:
# text = ["PTWORK", "CMMED"]

# REMOVE PTWORK FOR NOW
text = ["CMMED"]

In [219]:
cols_left = list(set(t.columns) - set(categorical) - set(text)  - set(["label", "Group","GROUP", "Phase", "RID", "PTID"]))
t[cols_left]

,NXPLANTA,NXGAIT,NXSENSOR,NXOTHER,PTDOBYY,PHC_LAN,NXMOTOR,PHC_VSP,NXNERVE,PHC_MEM,NXFINGER,PTTLANG,VISDATE,PTADDX,NXCONSCI,PTCOGBEG,SUBJECT_KEY,PTETHCAT,AGE,PHC_EXF,NXHEEL,PTRTYR,EXAMDATE,NXABNORM,PTRACCAT,NXTENDON
0,1.0,1.0,2.0,1.0,1933.0,1.813,1.0,0.264,1.0,1.530,1.0,1.0,2005-09-28,-4.0,1.0,-4.0,ADNI_011_S_0021,2.0,73.7467,0.401,1.0,1989.0,2006-11-01,1.0,4,1.0
1,1.0,1.0,2.0,1.0,1933.0,1.170,1.0,0.264,1.0,1.203,1.0,1.0,2005-09-28,-4.0,1.0,-4.0,ADNI_011_S_0021,2.0,74.7433,0.564,1.0,1989.0,2007-10-31,1.0,4,1.0
2,1.0,1.0,2.0,1.0,1933.0,1.372,1.0,-0.333,1.0,1.254,1.0,1.0,2005-09-28,-4.0,1.0,-4.0,ADNI_011_S_0021,2.0,73.2238,0.418,1.0,1989.0,2006-04-24,1.0,4,1.0
3,1.0,1.0,2.0,1.0,1933.0,0.816,1.0,0.264,1.0,1.300,1.0,1.0,2005-09-28,-4.0,1.0,-4.0,ADNI_011_S_0021,2.0,72.7255,0.288,1.0,1989.0,2005-10-24,1.0,4,1.0
4,1.0,1.0,2.0,1.0,1933.0,1.274,1.0,-4.000,1.0,1.620,1.0,1.0,2005-09-28,-4.0,1.0,-4.0,ADNI_011_S_0021,2.0,75.7207,0.645,1.0,1989.0,2008-10-22,1.0,4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2411,1.0,2.0,1.0,1.0,1926.0,-0.124,1.0,-0.804,1.0,-0.053,1.0,1.0,2012-04-03,9999.0,1.0,2009.0,ADNI_021_S_4659,2.0,88.9719,-0.348,1.0,1978.0,2015-04-21,1.0,5,1.0
2412,1.0,2.0,1.0,1.0,1926.0,0.152,1.0,-0.333,1.0,-0.026,1.0,1.0,2012-04-03,9999.0,1.0,2009.0,ADNI_021_S_4659,2.0,87.0226,-0.015,1.0,1978.0,2013-05-09,1.0,5,1.0
2413,1.0,2.0,1.0,1.0,1926.0,-0.091,1.0,0.483,1.0,0.083,1.0,1.0,2012-04-03,9999.0,1.0,2009.0,ADNI_021_S_4659,2.0,86.4969,-0.236,1.0,1978.0,2012-10-29,1.0,5,1.0
2414,1.0,2.0,1.0,1.0,1926.0,0.138,1.0,0.264,1.0,0.277,1.0,1.0,2012-04-03,9999.0,1.0,2009.0,ADNI_021_S_4659,2.0,85.9986,0.294,1.0,1978.0,2012-04-30,1.0,5,1.0


In [220]:
for col in cols_left:
    if len(t[col].value_counts()) < 10:
        print(col)
        categorical.append(col)

NXPLANTA
NXGAIT
NXSENSOR
NXOTHER
NXMOTOR
NXNERVE
NXFINGER
PTTLANG
PTADDX
NXCONSCI
PTETHCAT
NXHEEL
NXABNORM
PTRACCAT
NXTENDON


In [221]:
t.columns

Index(['RID', 'PTID', 'Phase', 'VISDATE', 'PTGENDER', 'PTDOBYY', 'PTHAND',
       'PTMARRY', 'PTEDUCAT', 'PTNOTRT', 'PTRTYR', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTCOGBEG', 'PTADDX', 'PTETHCAT', 'PTRACCAT', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 'NXOTHER',
       'NXABNORM', 'SUBJECT_KEY', 'EXAMDATE', 'AGE', 'PHC_MEM', 'PHC_EXF',
       'PHC_LAN', 'PHC_VSP', 'Group'],
      dtype='object')

In [222]:
# to_del = ["PTRTYR", "EXAMDATE", "SUBJECT_KEY", "PTWRECNT"]

#Remove PTWRECNT for now!! -> not in any .csv file
to_del = ["PTRTYR", "EXAMDATE", "SUBJECT_KEY"]
t = t.drop(to_del, axis=1)

In [223]:
quant = list(set(cols_left) - set(categorical) - set(text)  -set(to_del) - set(["label", "Group","GROUP", "Phase", "RID", "PTID"]))
t[quant]

,VISDATE,PTCOGBEG,AGE,PHC_EXF,PTDOBYY,PHC_LAN,PHC_VSP,PHC_MEM
0,2005-09-28,-4.0,73.7467,0.401,1933.0,1.813,0.264,1.530
1,2005-09-28,-4.0,74.7433,0.564,1933.0,1.170,0.264,1.203
2,2005-09-28,-4.0,73.2238,0.418,1933.0,1.372,-0.333,1.254
3,2005-09-28,-4.0,72.7255,0.288,1933.0,0.816,0.264,1.300
4,2005-09-28,-4.0,75.7207,0.645,1933.0,1.274,-4.000,1.620
...,...,...,...,...,...,...,...,...
2411,2012-04-03,2009.0,88.9719,-0.348,1926.0,-0.124,-0.804,-0.053
2412,2012-04-03,2009.0,87.0226,-0.015,1926.0,0.152,-0.333,-0.026
2413,2012-04-03,2009.0,86.4969,-0.236,1926.0,-0.091,0.483,0.083
2414,2012-04-03,2009.0,85.9986,0.294,1926.0,0.138,0.264,0.277


In [224]:
cols_left = list(set(cols_left) - set(categorical) - set(text) - set(quant) - set(to_del))

In [225]:
#after reviewing the meaning of each column, these are the final ones
# l = ['RID', 'PTID', 'Group', 'Phase', 'PTGENDER', 'PTDOBYY', 'PTHAND',
#        'PTMARRY', 'PTEDUCAT', 'PTWORK', 'PTNOTRT', 'PTHOME', 'PTTLANG',
#        'PTPLANG', 'PTCOGBEG', 'PTETHCAT', 'PTRACCAT', 'NXVISUAL',
#        'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
#        'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 
#        'NXABNORM',  'PHC_MEM', 'PHC_EXF', 'PHC_LAN', 'PHC_VSP']

# REMOVE PTWORK FOR NOW
l = ['RID', 'PTID', 'Group', 'Phase', 'PTGENDER', 'PTDOBYY', 'PTHAND',
       'PTMARRY', 'PTEDUCAT', 'PTNOTRT', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTCOGBEG', 'PTETHCAT', 'PTRACCAT', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 
       'NXABNORM',  'PHC_MEM', 'PHC_EXF', 'PHC_LAN', 'PHC_VSP']

In [226]:
t[l]

,RID,PTID,Group,Phase,PTGENDER,PTDOBYY,PTHAND,PTMARRY,PTEDUCAT,PTNOTRT,PTHOME,PTTLANG,PTPLANG,PTCOGBEG,PTETHCAT,PTRACCAT,NXVISUAL,NXAUDITO,NXTREMOR,NXCONSCI,NXNERVE,NXMOTOR,NXFINGER,NXHEEL,NXSENSOR,NXTENDON,NXPLANTA,NXGAIT,NXABNORM,PHC_MEM,PHC_EXF,PHC_LAN,PHC_VSP
0,21,011_S_0021,0.0,ADNI1,2.0,1933.0,1.0,3.0,18.0,1.0,1.0,1.0,1.0,-4.0,2.0,4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.530,0.401,1.813,0.264
1,21,011_S_0021,0.0,ADNI1,2.0,1933.0,1.0,3.0,18.0,1.0,1.0,1.0,1.0,-4.0,2.0,4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.203,0.564,1.170,0.264
2,21,011_S_0021,0.0,ADNI1,2.0,1933.0,1.0,3.0,18.0,1.0,1.0,1.0,1.0,-4.0,2.0,4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.254,0.418,1.372,-0.333
3,21,011_S_0021,0.0,ADNI1,2.0,1933.0,1.0,3.0,18.0,1.0,1.0,1.0,1.0,-4.0,2.0,4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.300,0.288,0.816,0.264
4,21,011_S_0021,0.0,ADNI1,2.0,1933.0,1.0,3.0,18.0,1.0,1.0,1.0,1.0,-4.0,2.0,4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.620,0.645,1.274,-4.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2411,4659,021_S_4659,2.0,ADNI2,1.0,1926.0,1.0,2.0,17.0,1.0,1.0,1.0,1.0,2009.0,2.0,5,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,-0.053,-0.348,-0.124,-0.804
2412,4659,021_S_4659,2.0,ADNI2,1.0,1926.0,1.0,2.0,17.0,1.0,1.0,1.0,1.0,2009.0,2.0,5,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,-0.026,-0.015,0.152,-0.333
2413,4659,021_S_4659,2.0,ADNI2,1.0,1926.0,1.0,2.0,17.0,1.0,1.0,1.0,1.0,2009.0,2.0,5,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,0.083,-0.236,-0.091,0.483
2414,4659,021_S_4659,2.0,ADNI2,1.0,1926.0,1.0,2.0,17.0,1.0,1.0,1.0,1.0,2009.0,2.0,5,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,0.277,0.294,0.138,0.264


In [227]:
dfs = []

In [228]:
for col in categorical:
    dfs.append(pd.get_dummies(t[col], prefix = col))

In [229]:
cat = pd.concat(dfs, axis=1)

In [230]:
t[quant]

,VISDATE,PTCOGBEG,AGE,PHC_EXF,PTDOBYY,PHC_LAN,PHC_VSP,PHC_MEM
0,2005-09-28,-4.0,73.7467,0.401,1933.0,1.813,0.264,1.530
1,2005-09-28,-4.0,74.7433,0.564,1933.0,1.170,0.264,1.203
2,2005-09-28,-4.0,73.2238,0.418,1933.0,1.372,-0.333,1.254
3,2005-09-28,-4.0,72.7255,0.288,1933.0,0.816,0.264,1.300
4,2005-09-28,-4.0,75.7207,0.645,1933.0,1.274,-4.000,1.620
...,...,...,...,...,...,...,...,...
2411,2012-04-03,2009.0,88.9719,-0.348,1926.0,-0.124,-0.804,-0.053
2412,2012-04-03,2009.0,87.0226,-0.015,1926.0,0.152,-0.333,-0.026
2413,2012-04-03,2009.0,86.4969,-0.236,1926.0,-0.091,0.483,0.083
2414,2012-04-03,2009.0,85.9986,0.294,1926.0,0.138,0.264,0.277


In [231]:
cat

,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTHOME_-4.0,PTHOME_1.0,PTHOME_2.0,PTHOME_3.0,PTHOME_4.0,PTHOME_5.0,PTHOME_6.0,PTHOME_8.0,PTMARRY_-4.0,PTMARRY_1.0,PTMARRY_2.0,PTMARRY_3.0,PTMARRY_4.0,PTMARRY_5.0,PTEDUCAT_-4.0,PTEDUCAT_7.0,PTEDUCAT_8.0,PTEDUCAT_10.0,PTEDUCAT_11.0,PTEDUCAT_12.0,PTEDUCAT_13.0,PTEDUCAT_14.0,PTEDUCAT_15.0,PTEDUCAT_16.0,PTEDUCAT_17.0,PTEDUCAT_18.0,PTEDUCAT_19.0,PTEDUCAT_20.0,PTPLANG_-4.0,PTPLANG_1.0,PTPLANG_2.0,PTPLANG_3.0,NXVISUAL_1.0,NXVISUAL_2.0,PTNOTRT_-4.0,PTNOTRT_0.0,PTNOTRT_1.0,PTNOTRT_2.0,NXTREMOR_1.0,NXTREMOR_2.0,NXAUDITO_1.0,NXAUDITO_2.0,PTHAND_-4.0,PTHAND_1.0,PTHAND_2.0,NXPLANTA_1.0,NXPLANTA_2.0,NXGAIT_-4.0,NXGAIT_1.0,NXGAIT_2.0,NXSENSOR_1.0,NXSENSOR_2.0,NXOTHER_-4.0,NXOTHER_-1.0,NXOTHER_1.0,NXOTHER_2.0,NXMOTOR_1.0,NXMOTOR_2.0,NXNERVE_1.0,NXNERVE_2.0,NXFINGER_1.0,NXFINGER_2.0,PTTLANG_-4.0,PTTLANG_1.0,PTTLANG_2.0,PTADDX_-4.0,PTADDX_2010.0,PTADDX_2011.0,PTADDX_9999.0,NXCONSCI_1.0,NXCONSCI_2.0,PTETHCAT_-4.0,PTETHCAT_1.0,PTETHCAT_2.0,PTETHCAT_3.0,NXHEEL_-4.0,NXHEEL_1.0,NXHEEL_2.0,NXABNORM_-4.0,NXABNORM_1.0,NXABNORM_2.0,PTRACCAT_-4,PTRACCAT_1,PTRACCAT_2,PTRACCAT_3,PTRACCAT_4,PTRACCAT_5,PTRACCAT_6,PTRACCAT_7,NXTENDON_1.0,NXTENDON_2.0
0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0
1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0
2,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0
3,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0
4,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2411,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,1,0,0,1,0,1,0,0,0,1,1,0,0,0,1,0,1,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0
2412,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,1,0,0,1,0,1,0,0,0,1,1,0,0,0,1,0,1,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0
2413,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,1,0,0,1,0,1,0,0,0,1,1,0,0,0,1,0,1,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0
2414,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,1,0,0,1,0,1,0,0,0,1,1,0,0,0,1,0,1,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0


In [232]:
t[["PTID","RID", "Phase", "Group"]]

,PTID,RID,Phase,Group
0,011_S_0021,21,ADNI1,0.0
1,011_S_0021,21,ADNI1,0.0
2,011_S_0021,21,ADNI1,0.0
3,011_S_0021,21,ADNI1,0.0
4,011_S_0021,21,ADNI1,0.0
...,...,...,...,...
2411,021_S_4659,4659,ADNI2,2.0
2412,021_S_4659,4659,ADNI2,2.0
2413,021_S_4659,4659,ADNI2,2.0
2414,021_S_4659,4659,ADNI2,2.0


In [233]:
c = pd.concat([t[["PTID", "RID", "Phase", "Group"]].reset_index(), cat.reset_index(), t[quant].reset_index()], axis=1).drop("index", axis=1) #tex

In [234]:
c

,PTID,RID,Phase,Group,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTHOME_-4.0,PTHOME_1.0,PTHOME_2.0,PTHOME_3.0,PTHOME_4.0,PTHOME_5.0,PTHOME_6.0,PTHOME_8.0,PTMARRY_-4.0,PTMARRY_1.0,PTMARRY_2.0,PTMARRY_3.0,PTMARRY_4.0,PTMARRY_5.0,PTEDUCAT_-4.0,PTEDUCAT_7.0,PTEDUCAT_8.0,PTEDUCAT_10.0,PTEDUCAT_11.0,PTEDUCAT_12.0,PTEDUCAT_13.0,PTEDUCAT_14.0,PTEDUCAT_15.0,PTEDUCAT_16.0,PTEDUCAT_17.0,PTEDUCAT_18.0,PTEDUCAT_19.0,PTEDUCAT_20.0,PTPLANG_-4.0,PTPLANG_1.0,PTPLANG_2.0,PTPLANG_3.0,NXVISUAL_1.0,NXVISUAL_2.0,PTNOTRT_-4.0,PTNOTRT_0.0,PTNOTRT_1.0,PTNOTRT_2.0,NXTREMOR_1.0,NXTREMOR_2.0,NXAUDITO_1.0,NXAUDITO_2.0,PTHAND_-4.0,PTHAND_1.0,PTHAND_2.0,NXPLANTA_1.0,NXPLANTA_2.0,NXGAIT_-4.0,NXGAIT_1.0,NXGAIT_2.0,NXSENSOR_1.0,NXSENSOR_2.0,NXOTHER_-4.0,NXOTHER_-1.0,NXOTHER_1.0,NXOTHER_2.0,NXMOTOR_1.0,NXMOTOR_2.0,NXNERVE_1.0,NXNERVE_2.0,NXFINGER_1.0,NXFINGER_2.0,PTTLANG_-4.0,PTTLANG_1.0,PTTLANG_2.0,PTADDX_-4.0,PTADDX_2010.0,PTADDX_2011.0,PTADDX_9999.0,NXCONSCI_1.0,NXCONSCI_2.0,PTETHCAT_-4.0,PTETHCAT_1.0,PTETHCAT_2.0,PTETHCAT_3.0,NXHEEL_-4.0,NXHEEL_1.0,NXHEEL_2.0,NXABNORM_-4.0,NXABNORM_1.0,NXABNORM_2.0,PTRACCAT_-4,PTRACCAT_1,PTRACCAT_2,PTRACCAT_3,PTRACCAT_4,PTRACCAT_5,PTRACCAT_6,PTRACCAT_7,NXTENDON_1.0,NXTENDON_2.0,VISDATE,PTCOGBEG,AGE,PHC_EXF,PTDOBYY,PHC_LAN,PHC_VSP,PHC_MEM
0,011_S_0021,21,ADNI1,0.0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,2005-09-28,-4.0,73.7467,0.401,1933.0,1.813,0.264,1.530
1,011_S_0021,21,ADNI1,0.0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,2005-09-28,-4.0,74.7433,0.564,1933.0,1.170,0.264,1.203
2,011_S_0021,21,ADNI1,0.0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,2005-09-28,-4.0,73.2238,0.418,1933.0,1.372,-0.333,1.254
3,011_S_0021,21,ADNI1,0.0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,2005-09-28,-4.0,72.7255,0.288,1933.0,0.816,0.264,1.300
4,011_S_0021,21,ADNI1,0.0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,2005-09-28,-4.0,75.7207,0.645,1933.0,1.274,-4.000,1.620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2411,021_S_4659,4659,ADNI2,2.0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,1,0,0,1,0,1,0,0,0,1,1,0,0,0,1,0,1,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,2012-04-03,2009.0,88.9719,-0.348,1926.0,-0.124,-0.804,-0.053
2412,021_S_4659,4659,ADNI2,2.0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,1,0,0,1,0,1,0,0,0,1,1,0,0,0,1,0,1,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,2012-04-03,2009.0,87.0226,-0.015,1926.0,0.152,-0.333,-0.026
2413,021_S_4659,4659,ADNI2,2.0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,1,0,0,1,0,1,0,0,0,1,1,0,0,0,1,0,1,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,2012-04-03,2009.0,86.4969,-0.236,1926.0,-0.091,0.483,0.083
2414,021_S_4659,4659,ADNI2,2.0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,1,0,0,1,0,1,0

In [235]:
#removing repeating subjects, taking the most recent diagnosis
c = c.groupby('PTID', 
                  group_keys=False).apply(lambda x: x.loc[x["Group"].astype(int).idxmax()]).drop("PTID", axis = 1).reset_index(inplace=False)

In [236]:
c.to_csv("clinical.csv")

In [237]:
#reading in the overlap test set
ts = pd.read_csv("overlap_test_set.csv").rename(columns={"subject": "PTID"})

#removing ids from the overlap test set
c = c[~c["PTID"].isin(list(ts["PTID"].values))]

FileNotFoundError: [Errno 2] No such file or directory: 'overlap_test_set.csv'

In [ ]:
cols = list(set(c.columns) - set(["PTID","RID","subject", "ID","GROUP", "Group", "label", "Phase", "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2", "update_stamp", "DX_x","DX_y", "Unnamed: 0"]))
X = c[cols].values 
y = c["Group"].astype(int).values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
X_train.to_pickle("X_train_c.pkl")
y_train.to_pickle("y_train_c.pkl")

X_test.to_pickle("X_test_c.pkl")
y_test.to_pickle("y_test_c.pkl")